# Service

Service 是將一組Pods的網絡端口公開的方法，

Kubernetes為Pod提供提供一個cluster IP,並以Load Balancer方式提供服務。

在之前的教學裡，我們使用port forwarding方式access Pods IP，
可是Pods IP是會隨Pods的生命週期而銷毀、改變。

我們無法知道下一次由ReplicaSet或Deployment建立的新的Pods的IP，
所以需要借助Service，幫我們找出對應的endpoints


In [ ]:
! kubectl delete deployment --all
! kubectl delete replicaset --all
! kubectl delete service --all
! kubectl delete pods --all

No resources found
No resources found
service "kubernetes" deleted
pod "hostname-82cwq" deleted
pod "hostname-thpsr" deleted


In [ ]:
! cat deployment.yaml

In [ ]:
! kubectl apply -f deployment.yaml

In [ ]:
! kubectl get deployment -o wide

NAME       READY   UP-TO-DATE   AVAILABLE   AGE   CONTAINERS   IMAGES                            SELECTOR
hostname   0/1     1            0           1s    hostname     docker.io/brokenpen/hostname:v1   app=hostname


In [ ]:
! kubectl expose deployment/hostname --type=NodePort

service/hostname exposed


In [ ]:
! kubectl get service hostname -o wide

NAME       TYPE       CLUSTER-IP       EXTERNAL-IP   PORT(S)          AGE   SELECTOR
hostname   NodePort   10.110.254.204   <none>        8080:32201/TCP   3s    app=hostname


In [ ]:
# 透過 minikube service 嘗試 access url
! minikube service hostname --url

😿  service default/hostname has no node port


In [ ]:
! curl $(minikube service hostname --url)

curl: (6) Could not resolve host: xn--638h
curl: (6) Could not resolve host: service
curl: (6) Could not resolve host: default
curl: (6) Could not resolve host: has
curl: (6) Could not resolve host: no
curl: (6) Could not resolve host: node
curl: (6) Could not resolve host: port


In [ ]:
! kubectl delete service hostname 
! kubectl expose deployment/hostname --port=8080 --target-port=8080

service "hostname" deleted
service/hostname exposed


In [ ]:
! kubectl get service hostname -o wide

NAME       TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)    AGE   SELECTOR
hostname   ClusterIP   10.98.197.220   <none>        8080/TCP   2s    app=hostname


In [ ]:
! kubectl get service hostname --no-headers -o custom-columns=":spec.clusterIP"

10.98.197.220


In [ ]:
! minikube ssh curl 10.98.197.220

🤷  The control plane node "" does not exist.
👉  To start a cluster, run: "minikube start"


In [ ]:
! kubectl get service hostname -o yaml

apiVersion: v1
kind: Service
metadata:
  creationTimestamp: "2022-11-28T06:46:26Z"
  name: hostname
  namespace: default
  resourceVersion: "1120"
  uid: 100a745d-95b8-4435-bc93-0390c5f26d1a
spec:
  clusterIP: 10.110.254.204
  clusterIPs:
  - 10.110.254.204
  externalTrafficPolicy: Cluster
  internalTrafficPolicy: Cluster
  ipFamilies:
  - IPv4
  ipFamilyPolicy: SingleStack
  ports:
  - nodePort: 32201
    port: 8080
    protocol: TCP
    targetPort: 8080
  selector:
    app: hostname
  sessionAffinity: None
  type: NodePort
status:
  loadBalancer: {}


In [ ]:
! kubectl delete service/hostname

service "hostname" deleted


In [ ]:
! kubectl get service hostname

Error from server (NotFound): services "hostname" not found


In [ ]:
! cat service.yaml

apiVersion: v1
kind: Service
metadata:
  name: hostname
spec:
  ports:
  - port: 8080
    protocol: TCP
    targetPort: 8080
  selector:
    app: hostname


In [ ]:
! kubectl apply -f service.yaml

service/hostname created


In [ ]:
! kubectl get service hostname -o wide

NAME       TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)    AGE   SELECTOR
hostname   ClusterIP   10.100.15.19   <none>        8080/TCP   2s    app=hostname


In [ ]:
! kubectl get po -o wide 

NAME                        READY   STATUS    RESTARTS   AGE    IP           NODE       NOMINATED NODE   READINESS GATES
hostname-85d476d6d4-pblrk   1/1     Running   0          119s   172.17.0.6   minikube   <none>           <none>


In [ ]:
! kubectl get po -o=custom-columns='IP:.status.podIP' --no-headers

172.17.0.6


In [ ]:
! curl $(kubectl get po -o=custom-columns='IP:.status.podIP' --no-headers):8080

curl: (7) Failed to connect to 172.17.0.6 port 8080 after 3050 ms: No route to host


In [ ]:
! kubectl get service hostname -o=custom-columns='IP:.spec.clusterIP' --no-headers

10.105.4.232


In [ ]:
! curl $(kubectl get service hostname -o=custom-columns='IP:.spec.clusterIP' --no-headers)

^C


In [ ]:
! curl 10.105.4.232:8080

^C


In [ ]:
! kubectl delete deploy --all
! kubectl delete replicaset --all
! kubectl delete service --all
! kubectl delete pod --all

No resources found
No resources found
service "kubernetes" deleted
No resources found


In [ ]:
! kubectl get endpoints hostname

NAME       ENDPOINTS         AGE
hostname   172.17.0.2:8080   92s


In [ ]:
! curl 172